In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType, FloatType, IntegerType, DateType, LongType
from pyspark.sql.catalog import Catalog
from os.path import abspath
import pandas as pd
sampledata = abspath('/mnt/d/linux/datasource/data/flight-data/csv')

spark = SparkSession.\
            builder.\
            master('local[*]').\
            appName('spark_functions').\
        getOrCreate()

In [ ]:
sampleschema = StructType([
                            StructField('DEST_COUNTRY_NAME', StringType(), True),
                            StructField('ORIGIN_COUNTRY_NAME', StringType(), True),
                            StructField('count', IntegerType(), True)
                        
                        ])


sampleflight = spark.read.\
                    csv(f'{sampledata}/*.csv',
                        header=True,
                        schema= sampleschema,
                        ).\
                    withColumnRenamed('DEST_COUNTRY_NAME', 'destine').\
                    withColumnRenamed('ORIGIN_COUNTRY_NAME', 'origin')
                    
sampleflight.createOrReplaceTempView("vflight")

In [ ]:
def totalcount(view:str, columns:str, quantity:str, country: str): 
    
    """ This function calculates total flights by country """
    
    queryresult = spark.sql(f"""
                                select 
                            
                                    {columns},
                                    sum({quantity}) as flightcount
                                  
                                from {view}
                                group by {columns}
                                having {columns} like "{country}"
                                
                            """)
    
    return queryresult
    
# totalcount('vflight', 'origin', 'count', 'India').show()
unitedflight = totalcount('vflight', 'origin', 'count', 'United States')

# unitedflightº


In [ ]:
totalfunction = udf(totalcount)

frametable = spark.table('vflight')

indiacount = frametable.select(totalfunction('frametable', 'origin', 'count', 'inidia'))
indiacount.show()

In [ ]:
def flightcount(frame:str, columns:str, quantity:str, country: str):
    
    """ function that calculate flights by country """
    
    return spark.sql( 
              f""" 
              
              select
              
                {columns}, 
                sum({quantity}
                
                from {frame}
                where {columns} = '{country}'
                group by {columns})
                
              """
            )

In [ ]:
spark.udf.register('flightcount', flightcount)

indiacount = spark.sql(""" select flightcount(vflight, origin, count, india) """)


In [ ]:
def rating(coluna:int):
    
    if coluna > 2000:
        
        return 'Classe A'
        
    elif coluna > 1500 and coluna < 2000:
        
        return 'Classe B'

    else:
        return 'Classe C'

In [ ]:
spark.udf.register('rating', rating, StringType())

ratefly = (sampleflight.select('origin', 'count').\
                        groupBy('origin').\
                        agg({'count': 'sum'}).\
                        withColumnRenamed('sum(count)', 'total').\
                        createOrReplaceTempView('rate'))

samplerate = spark.sql(""" select * from rate""")

rateoffly = spark.sql("""
                    
       
                    select 
                      origin,
                      total,
                      rating(total) as rating
                    from rate
                    
                    """
                    )

# spark.sql(""" select * from rate """).show()
rateoffly.show(10)

In [ ]:

def newrating(coluna:int):
    
    if coluna > 2000:
        
        return 'Classe A'
        
    elif coluna > 1500 and coluna < 2000:
        
        return 'Classe B'

    else:
        return 'Classe C'

In [129]:
ratingudf = udf(newrating)

# newrate = spark.sql( """
       
#                     select 
                    
#                       origin,
#                       total,
#                       newrating(total) as flightrate
                      
#                     from rate
                    
#                     """ )

# newrate.show(4)

# samplerate.selectExpr("ratingudf(total)").show(10)

samplerate.select('total', ratingudf('total').\
                alias('rating')).\
            show(10)

+-----+--------+
|total|  rating|
+-----+--------+
| 2063|Classe A|
|  416|Classe C|
|   85|Classe C|
|  508|Classe C|
| 1321|Classe C|
|  408|Classe C|
|  739|Classe C|
| 1573|Classe B|
|  104|Classe C|
|   29|Classe C|
+-----+--------+
only showing top 10 rows

